In [1]:
include("helper.jl")
include("algorithms.jl")
using NPZ
using Plots

In [2]:
Threads.nthreads()

6

# Competition

In [3]:
f(N) = ceil(Int64, 11 * N * log(N))

f (generic function with 1 method)

# Number of iterations tests

In [4]:
@inline function houdayer_mixed_tricks(
        h::Matrix{Float64}, nb::Int64, nb_iter::Int64,
        x_star::Vector{Int8}, a::Float64, b::Float64, adj::BitMatrix, n0::Int64)
    
    cur_x1 = generate_x(nb)
    cur_x2 = generate_x(nb)
    
    overlap_vector = Vector{Float64}(undef, nb_iter)
    
    nb_votes = 20000
    
    for i = 1:(nb_iter-nb_votes)
        if mod(i, n0) == 0 && (!all(cur_x1 .== cur_x2))
            houdayer_step!(cur_x1, cur_x2, adj, N)
        end
        
        metropolis_step!(h, cur_x1, nb, false)
        metropolis_step!(h, cur_x2, nb, false)
        
        if x_star != nothing
            overlap_vector[i] = overlap(cur_x1, x_star)
        end
    end
    
    votes1 = Matrix{Int8}(undef, nb, nb_votes)
    votes2 = Matrix{Int8}(undef, nb, nb_votes)
    
    for (idx, i) = enumerate((nb_iter-nb_votes+1):nb_iter)
        if mod(i, n0) == 0 && (!all(cur_x1 .== cur_x2))
            houdayer_step!(cur_x1, cur_x2, adj, N)
        end
        
        metropolis_step!(h, cur_x1, nb, true)
        metropolis_step!(h, cur_x2, nb, true)
        
        votes1[:, idx] = cur_x1
        votes2[:, idx] = cur_x2
        
        
        overlap_vector[i] = overlap(cur_x1, x_star)
    end
    
    cur_x1 = majority_vote(votes1)
    cur_x2 = majority_vote(votes2)
    
    return hamiltonian(cur_x1, h) < hamiltonian(cur_x2, h) ? cur_x1 : cur_x2, overlap_vector
end

houdayer_mixed_tricks (generic function with 1 method)

In [8]:
# Number of nodes in graph
a = 5.9
b = 0.1
n0 = 1000

N = 2500
x_star = generate_x(N)
adj = generate_graph(x_star, a, b)

nb_exp = 6
nb_iter = f(N)

215162

In [9]:
@time begin
    
    overlap_hat = run_experiment(N, a, b,
        x_star, houdayer_mixed_tricks, nb_iter, nb_exp, n0)
    npzwrite("data/overlap_test_comp_$(N).npy", overlap_hat)
    
end

(6, 215162)262.040220 seconds (45.91 M allocations: 249.313 GiB, 32.96% gc time, 2.02% compilation time)


In [10]:
plot(overlap_hat)

LoadError: InterruptException:

# Runtimes tests

In [ ]:
# Number of nodes in graph
a = 5.9
b = 0.1
n0 = 1000

N = 2500
x_star = generate_x(N)
adj = generate_graph(x_star, a, b)

nb_exp = 6
nb_iter = f(N)

In [ ]:
@time begin

    x_hat_matrix = competition(adj, a, b, nb_iter, nb_exp, N)
    
end